In [4]:
from quanser.hardware import HIL
import time

try:
    # Connect to QUBE-Servo 3 USB
    board = HIL("qube_servo3_usb", "0")
    print("Connected to QUBE-Servo 3 USB")

    encoder_channels = [0, 1]  # 0 = arm, 1 = pendulum
    motor_channels   = [0]     # motor output

    # Reset motor
    board.write_analog(len(motor_channels), motor_channels, [0.0])

    print("Applying test voltage...")
    for k in range(200):
        voltage = 0.5  # constant voltage
        board.write_analog(len(motor_channels), motor_channels, [voltage])

        # Read encoder values
        buffer = [0.0] * len(encoder_channels)
        board.read_encoder(len(encoder_channels), encoder_channels, buffer)

        print(f"Step {k:03d} | Arm: {buffer[0]} | Pendulum: {buffer[1]} | V={voltage:.2f}")
        time.sleep(0.01)  # 10 ms delay

    # Stop motor
    board.write_analog(len(motor_channels), motor_channels, [0.0])
    print("Test finished, motor stopped.")

except Exception as e:
    print("Error during test:", e)

board.close()


Connected to QUBE-Servo 3 USB
Error during test: a bytes-like object is required, not 'int'


In [ ]:
import numpy as np
from quanser.hardware import HIL
card = HIL("qube_servo3_usb", "0")
try:
  channels = np.array([0, 3, 5], dtype=np.uint32)
  num_channels = len(channels)
  minima = np.array([-10, 0, -5], dtype=np.int32)
  maxima = np.array([+10, 5, +5], dtype=np.int32)
  card.set_analog_input_ranges(channels, num_channels, minima, maxima)
  # ...

finally:
  card.close()